In [15]:
import json
from ipynb.fs.full.nltk_utils import tokenize, stem, bag_of_words
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from ipynb.fs.full.model import NeuralNet

In [16]:
with open('intents.json','r') as f:
    intents = json.load(f)
#print(intents) 

all_words = [] #of the bag of words
tags = [] #the tags of patterns
xy = []

# loop through each sentence in our intents patterns

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w) #no array so not used append 
        xy.append((w, tag))
        
ignore_words = ['?','!','.',','] 
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words)) 
tags = sorted(set(tags))
#print(all_words)
#print(tags)

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)


# create training data

X_train = [] #input
y_train = [] #output

for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    
    label = tags.index(tag)
    y_train.append(label) #not one-hot (binary data) CrossEntropyLoss

X_train = np.array(x_train)
y_train = np.array(y_train)
 
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples
    
#hyperParameters
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
# print(input_size, len(all_words))
# print(output_size, tags)
learning_rate = 0.001
num_epochs = 1000
    
dataset = ChatDataset()
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
train_loader = DataLoader(dataset = dataset , batch_size=batch_size, shuffle=True, num_workers=0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # check gpu support    
model = NeuralNet(input_size, hidden_size, output_size).to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels =  labels.to(dtype=torch.long).to(device)
        #forward
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        #backward and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss:{loss.item():.4f}')
print(f'final loss: {loss.item():.4f}')    

data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags
    
}

FILE = "data.pth" #pth for pytorhch
torch.save(data,FILE) #serializes and saves to pkl file

print(f'training complete. file saved to {FILE}')


      


29 patterns
8 tags: ['delivery', 'funny', 'goodbye', 'greeting', 'items', 'order', 'payments', 'thanks']
57 unique stemmed words: ["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'cup', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'some', 'someth', 'take', 'tea', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']
Epoch [100/1000], Loss:0.7685
Epoch [200/1000], Loss:1.0197
Epoch [300/1000], Loss:0.0613
Epoch [400/1000], Loss:0.0187
Epoch [500/1000], Loss:0.0221
Epoch [600/1000], Loss:0.0042
Epoch [700/1000], Loss:0.0070
Epoch [800/1000], Loss:0.0012
Epoch [900/1000], Loss:0.0006
Epoch [1000/1000], Loss:0.0004
final loss: 0.0004
training complete. file saved to data.pth
